# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [5]:
import pandas as pd

# Загрузка данных из файла
recipes = pd.read_csv('recipes_sample.csv', index_col=False)

# Выбор 5 случайных рецепфтов
random_recipes = recipes[['id', 'minutes']].sample(n=5, random_state=1)

# Функция для форматирования и печати таблицы
def print_table(dataframe):
    # Вычисляем ширину столбцов
    max_id_width = max(dataframe['id'].astype(str).apply(len).max(), len('id'))
    max_minutes_width = max(dataframe['minutes'].astype(str).apply(len).max(), len('minutes'))
    
    # Форматирование заголовка
    header = f"| {'id'.center(max_id_width)} | {'minutes'.center(max_minutes_width)} |"
    separator = f"|{'-' * (max_id_width + 2)}|{'-' * (max_minutes_width + 2)}|"
    
    # Форматирование строк
    rows = [
        f"| {str(row['id']).center(max_id_width)} | {str(row['minutes']).center(max_minutes_width)} |"
        for _, row in dataframe.iterrows()
    ]
    
    # Печать таблицы
    print(header)
    print(separator)
    for row in rows:
        print(row)

# Печать таблицы с 5 случайными рецептами
print_table(random_recipes)


|   id   | minutes |
|--------|---------|
| 78715  |   160   |
| 174859 |    65   |
| 301114 |    70   |
| 516384 |    35   |
| 95541  |   106   |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [6]:
import xml.etree.ElementTree as ET

tree = ET.parse('steps_sample.xml')
root = tree.getroot()

def show_info(recipe_id):
    # Поиск рецепта в данных
    recipe = recipes[recipes['id'] == recipe_id].iloc[0]
    
    # Поиск шагов в XML
    steps = []
    for recipe_elem in root.findall('recipe'):
        if int(recipe_elem.find('id').text) == recipe_id:
            for step in recipe_elem.find('steps').findall('step'):
                steps.append(step.text)
            break
    
    # Формирование строки с описанием
    recipe_info = f"{recipe['name']}\n\n"
    for i, step in enumerate(steps, 1):
        recipe_info += f"{i}. {step}\n"
    recipe_info += "----------\n"
    recipe_info += f"Автор: {recipe['contributor_id']}\n"
    recipe_info += f"Среднее время приготовления: {recipe['minutes']} минут"
    
    return recipe_info

# Вызов функции для рецепта с id 170895 и вывод на экран
recipe_description = show_info(170895)
print(recipe_description)


leeks and parsnips  sauteed or creamed

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an additi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [7]:
import re
import pandas as pd
import xml.etree.ElementTree as ET

# Загрузка данных из файла recipes_sample.csv
recipes = pd.read_csv('recipes_sample.csv', index_col=False)

# Загрузка данных из файла steps_sample.xml
tree = ET.parse('steps_sample.xml')
root = tree.getroot()

# Регулярное выражение для поиска числа и единиц времени
time_pattern = re.compile(r'\b(\d+)\s+(hour|hours|minute|minutes)\b')

# Поиск шагов для рецепта с id 25082
recipe_id = 25082
steps = []
for recipe_elem in root.findall('recipe'):
    if int(recipe_elem.find('id').text) == recipe_id:
        for step in recipe_elem.find('steps').findall('step'):
            steps.append(step.text)
        break

# Поиск паттернов в каждом шаге
results = []
for step in steps:
    matches = time_pattern.findall(step)
    if matches:
        results.extend(matches)

# Вывод всех непустых результатов с числами
for result in results:
    print(' '.join(result))


20 minutes
10 minutes
2 hours
10 minutes
20 minutes
30 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [8]:
import pandas as pd

# Загрузка данных из файла
recipes = pd.read_csv('recipes_sample.csv', index_col=False)

# Регулярное выражение для поиска шаблона
pattern = re.compile(r'^this\.\.\.,[\w\s]*but', re.IGNORECASE)

# Функция для поиска шаблона в строке
def find_pattern(description):
    if pd.isna(description):
        return False
    return bool(pattern.match(description))

# Применение функции к столбцу 'description'
matching_recipes = recipes[recipes['description'].apply(find_pattern)]

# Количество подходящих рецептов
count_matching_recipes = matching_recipes.shape[0]

# Три примера подходящих описаний
examples = matching_recipes['description'].head(3).tolist()

# Вывод результатов
print(f"Количество подходящих рецептов: {count_matching_recipes}")
print("Три примера подходящих описаний:")
for example in examples:
    print(f"- {example}")
#Такого рецепта нет??

Количество подходящих рецептов: 0
Три примера подходящих описаний:


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [9]:
tree = ET.parse('steps_sample.xml')
root = tree.getroot()

# Регулярное выражение для удаления пробелов вокруг символа дроби
fraction_pattern = re.compile(r'\s*/\s*')

# Поиск шагов для рецепта с id 72367
recipe_id = 72367
steps = []
for recipe_elem in root.findall('recipe'):
    if int(recipe_elem.find('id').text) == recipe_id:
        for step in recipe_elem.find('steps').findall('step'):
            steps.append(step.text)
        break

# Удаление пробелов вокруг символа дроби в каждом шаге
modified_steps = [fraction_pattern.sub('/', step) for step in steps]

# Вывод шагов после их изменения
print("Шаги рецепта с id 72367 после изменения:")
for step in modified_steps:
    print(step)


Шаги рецепта с id 72367 после изменения:
mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [10]:
from nltk.tokenize import word_tokenize

# Загрузка данных из файла steps_sample.xml
tree = ET.parse('steps_sample.xml')
root = tree.getroot()

# Сбор всех шагов рецептов в один список
all_steps = []
for recipe_elem in root.findall('recipe'):
    for step in recipe_elem.find('steps').findall('step'):
        all_steps.append(step.text)

# Токенизация и фильтрация только алфавитных слов, приведение к нижнему регистру
all_words = []
for step in all_steps:
    words = word_tokenize(step)
    words = [word.lower() for word in words if word.isalpha()]
    all_words.extend(words)

# Подсчет уникальных слов
unique_words = set(all_words)
print(f"Количество уникальных слов: {len(unique_words)}")


Количество уникальных слов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [11]:
from nltk.tokenize import sent_tokenize

# Загрузка данных из файла recipes_sample.csv
recipes = pd.read_csv('recipes_sample.csv', index_col=False)

# Функция для подсчета количества предложений в описании
def count_sentences(description):
    if pd.isna(description):
        return 0
    sentences = sent_tokenize(description)
    return len(sentences)

# Применение функции к столбцу 'description'
recipes['sentence_count'] = recipes['description'].apply(count_sentences)

# Нахождение 5 самых длинных описаний по количеству предложений
longest_descriptions = recipes.nlargest(5, 'sentence_count')

# Вывод строк фрейма, соответствующих этим рецептам
print(longest_descriptions[['id', 'description', 'sentence_count']])


           id                                        description  \
18408  334113  this wonderful icing is used for icing cakes a...   
481    287008  a translucent golden-brown crust allows the gr...   
22566  328708  this is one of the best soups i've ever made a...   
6779   205348  i wrote this because there are an astounding l...   
16296  316000  the first time i made this cake i grated a mil...   

       sentence_count  
18408              76  
481                27  
22566              24  
6779               23  
16296              23  


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [12]:
from nltk import pos_tag

# Функция для определения частей речи и форматирования вывода
def print_pos_tags(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    
    # Определение максимальной длины слова для выравнивания
    max_word_length = max(len(word) for word, _ in pos_tags)
    
    # Форматирование вывода
    pos_line = ""
    word_line = ""
    for word, pos in pos_tags:
        pos_line += f"{pos:^{max_word_length}} "  # Центрируем POS над словом
        word_line += f"{word:<{max_word_length}} "  # Левый отступ для слов
    
    print(pos_line.strip())  # Убираем последний пробел
    print(word_line.strip())  # Убираем последний пробел

# Проверка функции на названии рецепта с id 241106
recipe_id = 241106
recipe_name = recipes[recipes['id'] == recipe_id]['name'].values[0]
print_pos_tags(recipe_name)


JJ        NNS       IN        NNS       VBP       JJ        CC        JJ        NNS
eggplant  steaks    with      chickpeas feta      cheese    and       black     olives
